Install Dependencies

In [2]:
import numpy as np
import keras 
import tensorflow as tf
import os
from keras import backend as K
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from tensorflow.keras.models import load_model
from tensorflow.python.framework import graph_util
from tensorflow.python.framework.graph_util import convert_variables_to_constants

from PIL import Image
from tensorflow.keras.applications import DenseNet121

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.densenet import preprocess_input

import shutil

from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances

from keras.callbacks import Callback

from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

from sklearn.metrics import f1_score

#for classification report
from sklearn.metrics import classification_report
from keras.applications.mobilenet_v2 import preprocess_input

#for anomaly detection
from tensorflow.lite.python import interpreter as interpreter_wrapper
from sklearn.ensemble import IsolationForest
import cv2

ModuleNotFoundError: No module named 'numpy'

GPU ACTIVATION

In [ ]:
# Set the path to the CUDA installation directory
cuda_path = "/usr/local/cuda"  # Modify this path if necessary

# Set the environment variables for CUDA
os.environ["CUDA_HOME"] = cuda_path
os.environ["PATH"] = os.environ["PATH"] + ":" + cuda_path + "/bin"
os.environ["LD_LIBRARY_PATH"] = cuda_path + "/lib64"

# Specify which GPU devices to use (optional)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Set to "0,1" to use multiple GPUs

# Test if CUDA is available
import tensorflow as tf

if tf.test.is_gpu_available():
    print("CUDA is successfully activated!")
else:
    print("CUDA is not available or not properly configured.")

DATASET DIRECTORY


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input  # Preprocessing function

# Set paths
train_path = r'D:\EDUCATION FILES\fifth year\Thesis\REVISION FOR MAIN OBJECTIVE\1 SUPER FINAL\Final dataset 11\Train'
valid_path = r'D:\EDUCATION FILES\fifth year\Thesis\REVISION FOR MAIN OBJECTIVE\1 SUPER FINAL\Final dataset 11\Validation'
test_path = r'D:\EDUCATION FILES\fifth year\Thesis\REVISION FOR MAIN OBJECTIVE\1 SUPER FINAL\Final dataset 11\Test'

# Set image preprocessing parameters
input_size = (224, 224)
train_batch_size = 32
valid_batch_size = 32
test_batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Use the model's preprocessing function
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Vary brightness between 80% and 120%
    channel_shift_range=30  # Randomly shift channel values by up to 30
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Use the model's preprocessing function
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Ensure consistency in preprocessing

# Prepare the data generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=input_size,
    batch_size=train_batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=input_size,
    batch_size=valid_batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=input_size,
    batch_size=test_batch_size,
    class_mode='categorical'
)


MODEL ARCHITECTURE

In [ ]:
from tensorflow.keras.regularizers import l1, l2, l1_l2

# Set image preprocessing parameters
input_size = (224, 224)
train_batch_size = 32
valid_batch_size = 32
test_batch_size = 32

# Load MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

def create_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)  # L2 regularization
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu', kernel_regularizer=l1(0.01))(x)  # L1 regularization
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(x)  # Both L1 and L2 regularization
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    preds = Dense(6, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=preds)
    return model

model = create_model(base_model)

loss = 'categorical_crossentropy'
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

# Learning rate scheduling
def learning_rate_scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(learning_rate_scheduler)
batch_size = 32


DATASET DISTRIBUTION

In [ ]:
import os
import matplotlib.pyplot as plt

# Specify the path to the dataset directory
dataset_directory = r'D:\EDUCATION FILES\fifth year\Thesis\REVISION FOR MAIN OBJECTIVE\1 SUPER FINAL\Final dataset 11'

# Create an empty dictionary to store classifier counts
classifier_counts = {}

# Define the subfolders
subfolders = ['Train', 'Validation', 'Test']

# Iterate through each subfolder
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_directory, subfolder)
    classifiers = os.listdir(subfolder_path)
    
    # Iterate through each classifier
    for classifier in classifiers:
        classifier_path = os.path.join(subfolder_path, classifier)
        
        # Count the number of images in the classifier folder
        image_count = len([file for file in os.listdir(classifier_path) if os.path.isfile(os.path.join(classifier_path, file))])
        
        # Update the classifier counts dictionary
        if classifier not in classifier_counts:
            classifier_counts[classifier] = 0
        classifier_counts[classifier] += image_count

# Create a pie chart for all classifiers with total image counts and percentages
labels = list(classifier_counts.keys())
sizes = list(classifier_counts.values())
colors = plt.cm.Paired(range(len(labels)))
total_images = sum(sizes)
percentages = [(size / total_images) * 100 for size in sizes]

# Prepare labels for the pie chart with total images and percentages
labels_with_count = ["{} ({})".format(labels[i], sizes[i]) for i in range(len(labels))]
labels_with_percentage = ["{} ({:.1f}%)".format(labels[i], percentages[i]) for i in range(len(labels))]

plt.figure(figsize=(3, 3))
plt.pie(sizes, labels=labels_with_percentage, colors=colors, startangle=90, autopct='%1.1f%%')
plt.legend(labels_with_count, loc="best")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title("Dataset Distribution by Classifier")
plt.show()


CLASS WEIGHTS

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
# Define the number of samples per class. Ensure that the class labels are accurate and correspond to the order in your data.
samples_per_class = {
   "atopic": 746,
     "benign_melanoma": 800,
     "malignant_melanoma": 927,
     "psoriasis": 2883,
     "tinea": 685,
     "unknown": 1230
 }

# samples_per_class = {
#     "benign_melanoma": 800,
#     "malignant_melanoma": 928,
#     "psoriasis": 2469,
#     "tinea": 761,
#     "unknown": 1492
# }

# Calculate the total number of samples.
total_samples = sum(samples_per_class.values())

# Compute the class weights.
class_weights = {class_label: (1 / num_samples) * (total_samples) / len(samples_per_class) 
                 for class_label, num_samples in samples_per_class.items()}

# If your training generator uses categorical encoding, you might need to map the class names to their respective indices.
label_to_index = {label: index for index, label in enumerate(samples_per_class.keys())}

# Map the class weights to the respective class indices.
class_weight_indices = {label_to_index[label]: weight for label, weight in class_weights.items()}



FIT THE MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime

# ... [rest of your imports and model definition]

# TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Early stopping definition
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

# Reduce learning rate on plateau definition
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.000001)

# Train the model with the callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=200,  # You can increase this if you want, EarlyStopping will take care of halting the training if needed.
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    class_weight=class_weight_indices,
    callbacks=[tensorboard_callback, early_stopping, reduce_lr]
)

MODEL EVALUATION

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime

# ... [rest of your imports and model definition]

# TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

print("TensorBoard logs will be saved in:", log_dir)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Set the data directory for the test dataset
test_path = r'D:\EDUCATION FILES\fifth year\Thesis\REVISION FOR MAIN OBJECTIVE\1 SUPER FINAL\Final dataset 11\Test'

# Preprocess the test images
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size= test_batch_size,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on the test data
test_steps = (test_generator.n + test_batch_size - 1) // test_batch_size  # Calculate the correct number of steps
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

In [ ]:
# Generate predictions for the test data
test_generator.reset()  # Reset the generator
predictions = model.predict(test_generator, steps=test_steps)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true classes
true_classes = test_generator.classes[:len(predicted_classes)]

# Generate the classification report
class_labels = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print('Classification Report:')
print(report)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Generate predictions for the test data
test_generator.reset()  # Reset the generator
predictions = model.predict(test_generator, steps=test_steps)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true classes
true_classes = test_generator.classes[:len(predicted_classes)]

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Display the confusion matrix as an image with numbers
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)

# Add numbers to the confusion matrix
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.show()


In [ ]:
save_directory = r'D:\EDUCATION FILES\fifth year\Thesis\REVISION FOR MAIN OBJECTIVE'

# Save the entire model (including architecture, weights, and optimizer state)
model.save(os.path.join(save_directory, 'Final dataset 11 (.92 test).h5'))

In [ ]:
from keras.models import load_model
import tensorflow as tf

In [ ]:
model = load_model ("Final dataset 11 (.92 test).h5")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
print("model converted")

In [ ]:
#save model
with open('Final dataset 11 (.92 test).tflite', 'wb') as f:
    f.write(tflite_model)

REMOVE HIGH LOSS VALUE IMAGES

In [ ]:
# save_directory = 'E:\FILES\Thesis\practice\Datasets\Additional Datasets\From internet'

# # Save the entire model (including architecture, weights, and optimizer state)
# model.save(os.path.join(save_directory, 'dataset 10 (.94 test).h5'))